In [27]:
#Download libraries
import re
from collections import defaultdict
import pandas as pd
import altair as alt
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from preprocessing import *
import time

In [28]:
#Download the data
full_data = pd.read_csv('../data/train_data.zip')
full_data.head()

,external_id,month,year,monthly_number_of_sessions,monthly_unique_sessions,monthly_repeated_sessions,monthly_avg_length_of_session,monthly_avg_light_activity,monthly_avg_moderate_activity,monthly_avg_vigorous_activity,...,avg_wind_9_10,avg_wind_10_11,avg_wind_11_12,avg_wind_12_above,perfect_days,unacast_session_count,hpi,state_and_local_amount_per_capita,state_amount_per_capita,local_amount_per_capita
0,1807558,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,37.0,418.67,0.116368,0.025516,0.090852
1,1806729,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,63.0,710.00,0.175774,0.020170,0.155604
2,MR00096403,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,15.0,241.24,0.112038,0.006893,0.105145
3,1806350,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,45.0,468.51,0.120088,0.003619,0.116469
4,FM00169477,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,83.0,326.05,0.132935,0.025298,0.107637


In [8]:
biba_games_df = pd.DataFrame()
biba_games_df = pd.concat([full_data.loc[:, 'monthly_number_of_sessions':'distance_to_nearest_bus_stop'],
                           full_data.loc[:, 'days_since_first_sess':'historic_snow']], axis = 1)

#extracting categorical features
categorical_features = biba_games_df.loc[:, biba_games_df.dtypes == "object"]

In [10]:
monthly_survey_df = dict_to_columns_df(categorical_features['monthly_survey'], 'question', 'avg_answer')
monthly_weekday_counts_df = dict_to_columns_df(categorical_features['monthly_weekday_counts'], 'weekday', 'count')

In [13]:
monthly_survey_df.isnull().sum()

monthly_travel         48042
monthly_condition      47567
monthly_cleanliness    47551
monthly_regular        48007
monthly_accessible     48024
monthly_revisit        47950
monthly_variety        48028
monthly_safety         47560
monthly_allages        48029
dtype: int64

In [14]:
monthly_survey_df.shape

(50120, 9)

In [15]:
monthly_weekday_counts_df.shape

(50120, 7)

In [16]:
monthly_weekday_counts_df.isnull().sum()

monthly_Monday       48005
monthly_Wednesday    48117
monthly_Thursday     48065
monthly_Friday       48001
monthly_Saturday     47418
monthly_Tuesday      48152
monthly_Sunday       47403
dtype: int64

In [17]:
48/50

0.96

In [20]:
monthly_survey_df['monthly_regular'].value_counts()

1.000000    1428
0.000000     506
0.500000     127
0.666667      21
0.666667       8
0.333333       7
0.333333       7
0.800000       3
0.750000       3
0.400000       1
0.285714       1
0.250000       1
Name: monthly_regular, dtype: int64

In [21]:
monthly_survey_df

,monthly_travel,monthly_condition,monthly_cleanliness,monthly_regular,monthly_accessible,monthly_revisit,monthly_variety,monthly_safety,monthly_allages
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
50115,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
full_data

,external_id,month,year,monthly_number_of_sessions,monthly_unique_sessions,monthly_repeated_sessions,monthly_avg_length_of_session,monthly_avg_light_activity,monthly_avg_moderate_activity,monthly_avg_vigorous_activity,...,avg_wind_9_10,avg_wind_10_11,avg_wind_11_12,avg_wind_12_above,perfect_days,unacast_session_count,hpi,state_and_local_amount_per_capita,state_amount_per_capita,local_amount_per_capita
0,1807558,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,37.0,418.67,0.116368,0.025516,0.090852
1,1806729,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,63.0,710.00,0.175774,0.020170,0.155604
2,MR00096403,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,15.0,241.24,0.112038,0.006893,0.105145
3,1806350,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,45.0,468.51,0.120088,0.003619,0.116469
4,FM00169477,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,83.0,326.05,0.132935,0.025298,0.107637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50115,MR00111485,2,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,318.0,276.18,0.094705,0.008126,0.086578
50116,Kyle Rec Kenner 4,2,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,8.0,39.0,296.86,0.163209,0.063809,0.099400
50117,1901693,2,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,666.57,0.175774,0.020170,0.155604
50118,FM00166877,2,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,18.0,277.48,0.140205,0.013219,0.126986
